В данном блокноте удаляю дубликаты session_id, причём те, у которых target_event = 0

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from warnings import filterwarnings

filterwarnings('ignore')

%store -r df

# для этого датафрейма 
#df = df.sample(frac=1).reset_index(drop=True)
#df = df.sample(n=50000).reset_index(drop=True)

res = df[~((df.duplicated(['session_id'])) & (df['target_event']==0) )]
for col in res.columns:
    res[col] = res[col].fillna('unknown')

## visit_date и visit_time
здесь делаю некоторые манипуляции с датой и временем: смотрю сколько было целевых действий за месяц и за час - оказывается, час и месяц влияют на целевое действие

In [2]:
%%time
res['visit_time'] = pd.to_datetime(res['visit_time'])
res['visit_hour'] = res['visit_time'].apply(lambda x: int(x.hour))
vc = res[res['target_event']==1]['visit_hour'].value_counts()

ranges_hour = [0, 1000, 2000, 3000, 4000]
cats_hour = [1, 2, 3, 4]
df_vc = pd.DataFrame(vc)
df_vc['hour'] = df_vc.index
df_vc = df_vc.rename(columns={'visit_hour': 'count'})
df_vc.reset_index(drop=True)
df_vc['cat'] = pd.cut(df_vc['count'], ranges_hour, labels=cats_hour)

res['cat_hour'] = res['visit_hour'].apply(lambda x: df_vc.loc[df_vc['hour'], 'cat'][x])

res['visit_date'] = pd.to_datetime(res['visit_date'])
res['visit_month'] = res['visit_date'].apply(lambda x: int(x.month))

vcm = res[res['target_event']==1]['visit_month'].value_counts()

ranges_month = [0, 5000, 6000, 7000, 8000]
cats_month = [1, 2, 3, 4]
df_vcm = pd.DataFrame(vcm)
df_vcm['month'] = df_vcm.index
df_vcm = df_vcm.rename(columns={'visit_month': 'count'})
df_vcm.reset_index(drop=True)
df_vcm['cat'] = pd.cut(df_vcm['count'], ranges_month, labels=cats_month)
res['cat_month'] = res['visit_month'].apply(lambda x: df_vcm.loc[df_vcm['month'], 'cat'][x])

res = res.drop(['visit_date', 'visit_month', 'visit_time', 'visit_hour'], axis=1)

CPU times: total: 11min 15s
Wall time: 11min 16s


## UTM_*
### utm_source

In [3]:
def foo_source(x):
    #print(x)
    try:
        t = df_vc_utmsource.loc[df_vc_utmsource['source'], 'cat'][x]
    except:
        t = 0
    return t

In [4]:
%%time
adv_soc_net = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw','gVRrcxiDQubJiljoTbGm']
res['adv_soc_net'] = res['utm_source'].apply(lambda x: 1 if x in adv_soc_net else 0)

range_source = [0, 500, 5000, 50000]
cat_source = [1, 2, 3]
vc_utmsource = res[res['target_event']==1]['utm_source'].value_counts()
df_vc_utmsource = pd.DataFrame(vc_utmsource)
df_vc_utmsource['source'] = df_vc_utmsource.index
df_vc_utmsource = df_vc_utmsource.rename(columns={'utm_source': 'count'})
df_vc_utmsource['cat'] = pd.cut(df_vc_utmsource['count'], range_source, labels=cat_source).astype(np.int8)

res['cat_source'] = res['utm_source'].apply(lambda x: foo_source(x))
# res = res.drop(['utm_source'], axis=1)

CPU times: total: 13min 13s
Wall time: 13min 15s


### utm_medium

In [5]:
utm_medium_type = ['organic', 'referral', '(none)']
res['utm_medium_type'] = res['utm_medium'].apply(lambda x: 0 if x in ['organic', 'referral', '(none)'] else 1)
# res = res.drop(['utm_medium'], axis=1)

### utm_adcontent

In [6]:
# оставлю, обработка OneHotEncoder

### utm_campaign

In [ ]:
# оставлю, обработка OneHotEncoder


In [25]:
res = res.drop(['session_id', 'client_id', 'device_model', 'visit_number', 'utm_source', 'utm_keyword'], axis=1)
res.head(10)

,utm_campaign,utm_adcontent,device_category,device_os,target_event,cat_hour,cat_month,adv_soc_net,cat_source,utm_medium_type,cat_os,cat_device_brand,cat_browser,is_russia,cat_city
0,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,0,4,3,0,3,1,1,2,1,1,0
1,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,mobile,Android,0,2,3,1,2,1,1,2,0,1,0
2,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,0,1,4,0,3,1,1,2,1,1,0
3,unknown,NOBKLgtuvqYWkXQHeYWM,mobile,Android,0,1,3,0,3,1,1,2,1,1,0
4,unknown,unknown,mobile,Android,0,1,3,0,3,1,1,2,1,1,0
5,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,iOS,0,4,3,0,3,0,1,3,1,1,0
6,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,tablet,Android,0,2,4,0,0,1,1,1,1,1,0
7,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,mobile,Android,0,3,1,0,3,1,1,2,1,1,0
8,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,iOS,0,4,4,0,3,1,1,3,1,1,0
9,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,mobile,Android,0,4,3,0,3,1,1,2,1,1,0


## DEVICE_*

### device_category

device_category - 3 категории, пропусков нет. Из преобразований - будет последующее onehot-кодирование

### device_os, device_brand, device_browser

восстановим ОС из других значений device_*

In [11]:
best_brands = ['Samsung', 'Xiaomi', 'Huawei', 'POCO']

dev_types = ['mobile', 'tablet']

android_brands = ['Xiaomi', 'Samsung', 'Huawei', 'Vivo', 'LG',
       'BQ', 'OnePlus', 'itel', 'Asus', 'Sony', 'Android', 'OPPO',
       'Nokia', 'Realme', 'Alcatel', 'Infinix', 'Micromax', 'Blackview',
       'Oukitel', 'ZTE', 'Motorola', 'HOMTOM', 'Google', 'Lenovo',
       'China Phone', 'Neffos', 'Tecno', 'DOOGEE', 'Vsmart', 'Meizu', 'Umidigi', 'Leagoo', 'Highscreen', 'Ulefone', 'HTC',
       'Cubot', 'Symphony', 'Vestel', 'Guophone', 'Wileyfox', 'Vertex',
       'POCO', 'TCL', 'SonyEricsson', 'Nomu', 'Prestigio',
       'Philips', 'LeEco', 'DEXP', 'Hisense', 'Vernee', 'Tele2', 'CAT',
       'H96', 'Kyocera', 'Inoi', 'Acer', 'Kingplay', 'Sharp',
       'MTC', 'iNew', 'Karbonn', 'Wiko', 'Gionee', 'Digma',
       'Yota Devices', 'Kata', 'Poptel', 'Flylion', 'AGM', 'Teclast',
       'Fly', 'MLS', 'LeTV', 'Nuu', 'Celkon', 'Chuwi', 'Walton',
       'Haier', 'Coolpad', 'Xiaolajiao', 'Lava', 'General Mobile',
       'Black Fox', 'Condor', 'Razer', 'Microsoft', 'Fujitsu',
       'Alldocube', 'ZOJI', 'Amazon', 'PPTV', 'Jiake', 'Mito', 'Oysters',
       'NOA', 'Caterpillar', 'SenseIT', 'Verizon', 'Komu', 'BLU', 'RCA',
       'Cube', 'AT&T', 'Tonbux', 'Dark', 'Elephone', 'T-Mobile',
       'Smartfren', 'Billion', 'Gome', 'A1', 'MTS', 'Geotel', 'BLUBOO',
       'Ananda', 'MediaTek', 'KingSing', 'Ark', 'Essential',
       'Dragon Touch', 'E&L', 'Fero',  'Wigor', 'Tanix',
       'Mlais', 'InFocus', 'Smartisan', 'Beelink', 'Unihertz', 'Orbic',
       'Honeywell', 'Archos', 'RED', 'Sonim', 'Jiayu', 'Evertek',
       'Leegoog', 'QMobile', 'INUI', 'Artel', 'Irbis', 'China Mobile',
       'Flexymove', 'Olla', 'Krip', 'Star', 'UMI', 'Iris', 'Ellipsis',
       'LTC', 'Motive', 'Advan', 'M-HORSE', 'Winnovo', 'ThL', 'Nomi',
       'Wings Mobile', 'MXQ', 'How', 'Centric', 'Maze', 'Corn',
       'China TVBox', 'Gretel', 'Vertu', 'Evercoss', 'teXet', 'Qbex',
       'Panasonic', 'Tagital', 'Dell', 'Maxvi']

In [12]:
%%time
def foo(x):
    if x['device_os']=='unknown':
        if x['device_brand'] in android_brands:
            return 'Android'
        elif (x['device_brand'] == 'Apple') and (x.device_category in dev_types):
            return 'iOS'
    elif x['device_os'] != x['device_os']:
        if (x['device_category'] in dev_types) and (x['device_brand']=='Apple'):
            return 'iOS'
        elif x['device_brand'] in android_brands:
            return 'Android'
    else:
        return x['device_os']
    return x['device_os']
    
res['device_os'] = res.apply(lambda x: foo(x), axis=1)

CPU times: total: 23.9 s
Wall time: 24 s


device_brand преобразуем в три категории: Apple - 3, Samsung, Xiaomi, Huawei и Poco - 2, остальные бренды - 1
признак device_browser отбросим - 

In [13]:
def foo_os(x):
    if x in ['Android', 'iOS']:
        return 1
    else:
        return 0
    return 0

res['cat_os'] = res['device_os'].apply(lambda x: foo_os(x))

In [14]:
def brand2cat(x):
    if x == 'Apple':
        return int(3)
    elif x in best_brands:
        return int(2)
    else:
        return int(1)
    return int(0)

res['cat_device_brand'] = res['device_brand'].apply(lambda x: brand2cat(x))
res = res.drop(['device_brand'], axis=1)

In [16]:
%%time
def foo_browser(x):
    if x in ['Chrome', 'Safari', 'YaBrowser']:
        return 1
    else:
        return 0
    return 0


res['cat_browser'] = res['device_browser'].apply(lambda x: foo_browser(x)) 

CPU times: total: 516 ms
Wall time: 525 ms


device_screen_resolution удалю

In [17]:
res = res.drop(['device_browser', 'device_screen_resolution'], axis=1)

## GEO_*
### geo_country

In [18]:
res['is_russia'] = res['geo_country'].apply(lambda x: 1 if x=='Russia' else 0)

### geo_city

In [19]:
def foo_city(x):
    #print(x)
    t = 0
    try:
        t = df_vc_city.loc[df_vc_browser['city'], 'cat'][x]
    except:
        t = 0
    return t

In [20]:
%%time
range_city = [0, 100, 1000, 10000, 100000]
cat_city = [0, 1, 2, 3]
vc_city = res[res['target_event']==1]['geo_city'].value_counts()
df_vc_city = pd.DataFrame(vc_city)
df_vc_city['city'] = df_vc_city.index
df_vc_city = df_vc_city.rename(columns={'geo_city': 'count'})
df_vc_city['cat'] = pd.cut(df_vc_city['count'], range_city, labels=cat_city)
res['cat_city'] = res['geo_city'].apply(lambda x: foo_city(x)) #.astype(np.int8)
# res = res.drop(['utm_campaign'], axis=1)

CPU times: total: 1.5 s
Wall time: 1.51 s


In [21]:
res = res.drop(['geo_country', 'geo_city'], axis=1)

In [22]:
def view_occupancy(df):
    length = len(df)
    dic = dict()
    for col in df.columns:
        percent_filled = (length-df[col].isna().sum())/length
        dic[col] = percent_filled

    dic = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    for k, v in dic:
        print('{}: {:.3f}%'.format(k, v*100))

In [26]:
# view_occupancy(res)

utm_campaign: 100.000%
utm_adcontent: 100.000%
device_category: 100.000%
device_os: 100.000%
target_event: 100.000%
cat_hour: 100.000%
cat_month: 100.000%
adv_soc_net: 100.000%
cat_source: 100.000%
utm_medium_type: 100.000%
cat_os: 100.000%
cat_device_brand: 100.000%
cat_browser: 100.000%
is_russia: 100.000%
cat_city: 100.000%


In [24]:
res.head()

,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,target_event,cat_hour,cat_month,adv_soc_net,cat_source,utm_medium_type,cat_os,cat_device_brand,cat_browser,is_russia,cat_city
0,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,0,4,3,0,3,1,1,2,1,1,0
1,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,0,2,3,1,2,1,1,2,0,1,0
2,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,0,1,4,0,3,1,1,2,1,1,0
3,unknown,NOBKLgtuvqYWkXQHeYWM,unknown,mobile,Android,0,1,3,0,3,1,1,2,1,1,0
4,unknown,unknown,unknown,mobile,Android,0,1,3,0,3,1,1,2,1,1,0


In [28]:
# res = res.sample(frac=1).reset_index(drop=True)
res = res.sample(n=200000).reset_index(drop=True)

cols_for_ohe = ['utm_medium', 'utm_campaign', 'utm_adcontent', 'device_category', 'device_os']

ohe = OneHotEncoder(sparse=False)

ohe.fit(res[cols_for_ohe])
transformed = ohe.transform(res[cols_for_ohe])
res[ohe.get_feature_names_out()] = transformed
df_ohed = res.drop(cols_for_ohe, axis=1)

%store df_ohed

Stored 'df_ohed' (DataFrame)
